In [1]:
%reload_ext tensorboard
from datetime import datetime
from tensorflow.core.protobuf import rewriter_config_pb2
from packaging import version
import tensorflow as tf
from tensorflow import keras
from __future__ import print_function
from keras import regularizers
import tensorflow.keras as keras
import tensorflow.keras.datasets as datasets
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
import random
import numpy as np
import sklearn.metrics as metrics
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time
import datetime, os

num_classes = 10

print("TensorFlow version: ", tf.__version__)

# input image dimensions
img_rows, img_cols = 32, 32


TensorFlow version:  2.6.0


In [2]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test_orig = y_test
y_test = keras.utils.to_categorical(y_test, num_classes)

# Uncomment these lines to enable the data augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False)
 #compute quantities required for featurewise normalization
 #(std, mean, and principal components if ZCA whitening is applied)
datagen.fit(x_train)



170508288/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
rm -rf ./logs #clears model logs

In [4]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))


model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))


model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

tensorboard = TensorBoard(log_dir="logs/final-model".format())

optimizer = keras.optimizers.RMSprop(lr=0.001, decay=1e-6)
  

model.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=optimizer,
            metrics=['accuracy'])
  
''' 
def train_model(lr):
  #This was used when running tests. 

  # resets the model 
  del model
  K.clear_session()
  tf.compat.v1.reset_default_graph()
  
for lr in [0.1, 0.01, 0.001, 0.0001]:
  train_model(lr)
'''  




/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


' \ndef train_model(lr):\n  #This was used when running tests. \n\n  # resets the model \n  del model\n  K.clear_session()\n  tf.compat.v1.reset_default_graph()\n  \nfor lr in [0.1, 0.01, 0.001, 0.0001]:\n  train_model(lr)\n'

In [ ]:
history = model.fit(x=x_train, 
                    y=y_train, 
                    epochs=170,
                    batch_size=64,
                    validation_data=(x_test, y_test), 
                    callbacks=[tensorboard])

predictions = model.predict(x_test)
predicted_classes = np.argmax(predictions, axis=1)

print(metrics.classification_report(y_test_orig, predicted_classes))
print()
print(metrics.confusion_matrix(y_test_orig, predicted_classes))

plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

Epoch 1/170
782/782 [==============================] - 59s 34ms/step - loss: 1.8210 - accuracy: 0.4673 - val_loss: 1.5376 - val_accuracy: 0.5119
Epoch 2/170
782/782 [==============================] - 22s 29ms/step - loss: 1.1727 - accuracy: 0.6443 - val_loss: 1.1168 - val_accuracy: 0.6669
Epoch 3/170
782/782 [==============================] - 23s 29ms/step - loss: 0.9542 - accuracy: 0.7074 - val_loss: 1.2699 - val_accuracy: 0.6002
Epoch 4/170
287/782 [==========>...................] - ETA: 13s - loss: 0.8571 - accuracy: 0.7383

In [ ]:
#view TensorBoard logs with graphs
%tensorboard --logdir logs/

In [ ]:
img_index = 42
image = x_test[img_index]

pred=model.predict(np.expand_dims(image, axis=0))[0]
for cl in range(num_classes):
        print("Probability for class {}: {}".format(cl,pred[cl]))
print("\nThe winner is {}".format(np.argmax(pred)))
print("The correct class is {}\n".format(np.argmax(y_test[img_index])))

plt.imshow(image.squeeze(),cmap='viridis')
plt.show()




